In [ ]:
# #필요한 파일 인스톨
# !pip install gym pyvirtualdisplay
# # • apt-get install -y xvfb python-opengl ffmpeg
# # •
# # apt-get update
# # • apt-get install cmake
# !pip install --upgrade setuptools
# !pip install ez_setup
# !pip install gym[atari]

# !pip install box2d-py
# !pip install gym[Box_2D]
# !pip install pyglet

## Setup

In [1]:
import numpy as np
import gym
import random as rnd
from gym import logger as gym_logger
gym_logger.set_level(40)
from gym.wrappers.record_video import RecordVideo
from tqdm import tqdm

import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import seaborn as sns

In [ ]:
def show_vedio():
    mp4list = glob.glob("video/*.mp4")
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, "r+w").read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data="''''".fotmat(encoded.decode("ascii"))))
    else:
        print("error")
    
def wrap_env(env):
    env = RecordVideo(env, "./video", episode_trigger=lambda x: True)
    env.reset()
    return env

## MDP with creating video data

In [ ]:
env = wrap_env(gym.make("MountainCar-v0", render_mode="rgb_array"))

step = 0
score = 0

while True:
    # random sampling action
    action = env.action_space.sample()
    # get result from action
    obs, reward, terminated, truncated, info = env.step(action)
    # terminated : 목표에 도달했는지에 대한 여부 
    # truncated : step에 도달했는지에 대한 여부
    score += reward
    step += 1
    if (terminated or truncated) or (step > 200):
        break

## MDP with DNN

### Create training dataset

In [2]:
env = gym.make("MountainCar-v0", render_mode="human")

n_steps = 1000
n_scores = []
training_data = []
accepted_scores = []
required_score = -1 * (n_steps * 1) + (n_steps * 1 * 1)
required_score

0

In [3]:
for i in tqdm(range(100)):
    if i % 100 == 0:
        print(i)
    env.reset()
    step = 0
    score = 0
    game_memory = []
    previous_obs = None
    while True:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        # save T-1 th position & T th action
        if previous_obs is not None:
            game_memory.append([previous_obs, action])
        previous_obs = obs
        # calculating reward from policy
        if obs[0] > -0.2:
            reward = 1
        score += reward
        if (terminated or truncated) or (step > n_steps):
            break
        else:
            step += 1
    # save raw score value
    n_scores.append(score)
    if score > required_score:
        # save best case score value
        accepted_scores.append(score)
        # save best case parameter for decision making
        for data in game_memory:
            training_data.append(data)
    # break


  0%|          | 0/10000 [00:00<?, ?it/s]

0


c:\Users\USER\Desktop\class_reinforcement_learning\.venv\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
  0%|          | 23/10000 [01:19<9:32:59,  3.45s/it]

In [11]:
n_scores = np.array(n_scores)
# print(n_scores.mean())
# print(accepted_scores)

train_x = np.array([i[0] for i in training_data]).reshape(-1, 2)
train_y = np.array([i[1] for i in training_data]).reshape(-1, 1)

In [13]:
model = Sequential([
    layers.Dense(4, activation="relu", input_shape=(2,)),
    layers.Dense(3, activation="softmax"),
])
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

filename = "model_rf.h5"

history = model.fit(train_x, train_y, epochs=5, batch_size=4, validation_split=0.2)

Epoch 1/5
40/40 [==============================] - 2s 5ms/step - loss: 1.0944 - accuracy: 0.3522 - val_loss: 1.1183 - val_accuracy: 0.3250
Epoch 2/5
40/40 [==============================] - 0s 2ms/step - loss: 1.0939 - accuracy: 0.3774 - val_loss: 1.1185 - val_accuracy: 0.3250
Epoch 3/5
40/40 [==============================] - 0s 2ms/step - loss: 1.0939 - accuracy: 0.3774 - val_loss: 1.1178 - val_accuracy: 0.3250
Epoch 4/5
40/40 [==============================] - 0s 2ms/step - loss: 1.0936 - accuracy: 0.3774 - val_loss: 1.1173 - val_accuracy: 0.3250
Epoch 5/5
40/40 [==============================] - 0s 2ms/step - loss: 1.0934 - accuracy: 0.3774 - val_loss: 1.1168 - val_accuracy: 0.3250
